In [1]:
!pip install pennylane
!git clone https://github.com/PennyLaneAI/pennylane-lightning.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
Cloning into 'pennylane-lightning'...
remote: Enumerating objects: 29304, done.
remote: Counting objects: 100% (3524/3524), done.
remote: Compressing objects: 100% (1121/1121), done.
remote: Total 29304 (delta 2786), reused 3036 (delta 2403), pack-reused 25780
Receiving objects: 100% (29304/29304), 8.43 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (22873/22873), done.


In [2]:
import os
os.chdir("pennylane-lightning")
!pip install -r requirements.txt
!PL_BACKEND="lightning_qubit" pip install -e . -vv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.7/234.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4
Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-e4s1_wmr
Initialized build tracking at /tmp/pip-build-tracker-e4s1_wmr
Created build tracker: /tmp/pip-build-tracker-e4s1_wmr
Entered build tracker: /tmp/pip-build-tracker-e4s1_wmr
Created

In [3]:
!python -m pip install wheel custatevec-cu11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 25.7 MB/s eta 0:00:00


In [4]:
import os
import site
os.environ["CUQUANTUM_SDK"] = f'{site.getsitepackages()[0]}/cuquantum/lib'

In [5]:
!PL_BACKEND="lightning_gpu" python -m pip install -e . -vv

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-5znkopcg
Initialized build tracking at /tmp/pip-build-tracker-5znkopcg
Created build tracker: /tmp/pip-build-tracker-5znkopcg
Entered build tracker: /tmp/pip-build-tracker-5znkopcg
Created temporary directory: /tmp/pip-install-6vmbh7p6
Created temporary directory: /tmp/pip-ephem-wheel-cache-km6mk59k
Obtaining file:///content/pennylane-lightning
  Added file:///content/pennylane-lightning to build tracker '/tmp/pip-build-tracker-5znkopcg'
  Running setup.py (path:/content/pennylane-lightning/setup.py) egg_info for package from file:///content/pennylane-lightning
  Created temporary directory: /tmp/pip-pip-egg-info-90ete8ae
  Running command python setup.py egg_info
  /usr/local/lib/python3.10/dist-packages/setuptools/dist.py:300: InformationOnly: Normalizing '0.38.0-dev26' to '0.38.0.dev26'
    self.metadata.

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.nn.functional as F

import pennylane as qml
from pennylane import numpy as np

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
n_qubits = 4
if use_cuda is True:
    dev = qml.device("lightning.gpu", wires=n_qubits)
else:
    dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

print(f"{dev=}")

input = torch.from_numpy(np.random.uniform(0, np.pi, (4,))).to(device)
weights = torch.from_numpy(np.random.uniform(0, np.pi, (4, 4))).to(device)
output = circuit(input, weights)
print(f"{output=}")

dev=<default.qubit device (wires=4) at 0x7cee3436bbe0>
output=[tensor(0.1058, dtype=torch.float64), tensor(-0.0932, dtype=torch.float64), tensor(-0.0573, dtype=torch.float64), tensor(-0.3310, dtype=torch.float64)]


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import json
# Opening JSON file
f = open('/content/drive/MyDrive/al5083/al5083/train/train.json',)
#f=open('/content/drive/MyDrive/alweld/train/train.json')
label_train = json.load(f)

In [10]:
keys_orig = list(label_train.keys())
values = list(label_train.values())
keys = []
for key in keys_orig:
  a_string = key
  a_string = a_string.replace("/", " ")
  x = a_string.split()
  string = x[0]+' '+x[1]
  keys.append(string)
true_labels_dict = {keys[i]: values[i] for i in range(len(keys))}
true_labels_dict

{'170906-113317-Al 2mm-part3': 1,
 '170906-144958-Al 2mm': 1,
 '170904-141730-Al 2mm-part2': 2,
 '170904-151845-Al 2mm-part2': 2,
 '170906-153326-Al 2mm-part2': 2,
 '170904-141232-Al 2mm-part2': 2,
 '170913-151508-Al 2mm-part2': 2,
 '170904-150144-Al 2mm-part1': 2,
 '170913-143933-Al 2mm-part2': 2,
 '170913-152931-Al 2mm-part2': 2,
 '170906-141809-Al 2mm-part2': 2,
 '170815-134756-Al 2mm': 2,
 '170904-115959-Al 2mm': 2,
 '170815-133921-Al 2mm': 2,
 '170904-112347-Al 2mm': 2,
 '170904-145718-Al 2mm-part2': 4,
 '170904-141730-Al 2mm-part3': 4,
 '170905-110711-Al 2mm-part2': 4,
 '170904-141232-Al 2mm-part3': 4,
 '170913-155806-Al 2mm-part2': 4,
 '170904-152301-Al 2mm-part1': 4,
 '170904-113012-Al 2mm-part2': 4,
 '170904-154202-Al 2mm-part2': 4,
 '170904-155523-Al 2mm': 4,
 '170913-140725-Al 2mm': 4,
 '170913-152931-Al 2mm-part1': 0,
 '170913-151508-Al 2mm-part1': 0,
 '170913-143933-Al 2mm-part1': 0,
 '170904-141730-Al 2mm-part1': 0,
 '170904-145718-Al 2mm-part1': 0,
 '170913-142501-Al 2mm

In [11]:
import numpy as np
import pandas as pd
import os
from os import walk

In [13]:
location_folders = '/content/drive/My Drive/al5083/al5083/train/'
folders = []
for (_, dirnames, _) in walk(location_folders):
    folders.extend(dirnames)
    break

num_files = []
for folder in folders:
  location_files = location_folders + folder +'/'
  files =[]
  for (_, _, filenames) in walk(location_files):
    files.extend(filenames)
    break

  num_files.append(len(files))

num_files = np.array(num_files)
num_files

array([189,  77, 135, 155, 352, 113, 125,  48,   4,  38,  26, 288,  78,
        17,  10,  37,  29,   4, 146, 151,  38, 106,  28,  80, 123, 115,
        71, 152, 130,  57,  97, 158, 289,  77, 790, 316,   7,  48, 336,
        26,  47, 117,  12,  27,  45,  56])

In [14]:
label = [true_labels_dict[folder] for folder in folders]
train_data = pd.DataFrame({'folders':folders,
                           'number_of_files':num_files,
                           'Labels':label })
train_data_detail =train_data.sort_values(by=['Labels'])

In [15]:
filename = 'data_detail.csv'
train_data_detail.to_csv(filename, index=False)

In [16]:
import os
from os import walk

location_folders = '/content/drive/My Drive/Dataset/al5083/al5083/train/'
#location_folders = '/content/drive/MyDrive/alweld/train'

folders = []
for (_, dirnames, _) in walk(location_folders):
    folders.extend(dirnames)
    break
# folders
for folder in folders:
  location_files = location_folders + folder +'/'
  files =[]
  for (_, _, filenames) in walk(location_files):
    files.extend(filenames)
    break
  # print(len(files))

#   """
#   Now we need to reduce the skewness of the data
#   Min is 55 and max goes to 2400
#   """
  if true_labels_dict[folder] == 0:
    for i,file in enumerate(files):
      if (i+1)%8 == 1:
        # print('Skipped successfully')
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)
  elif true_labels_dict[folder] == 1:
    for i,file in enumerate(files):
      if (i+1)%2 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)
  elif true_labels_dict[folder] == 2:
    for i,file in enumerate(files):
      if (i+1)%7 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)
  elif true_labels_dict[folder] == 3:
    for i,file in enumerate(files):
      if (i+1)%4 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)
  elif true_labels_dict[folder] == 4:
    for i,file in enumerate(files):
      if (i+1)%3 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)
  elif true_labels_dict[folder] == 5:
    for i,file in enumerate(files):
      if (i+1)%3 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)
        print("%s has been removed successfully" %file)

In [17]:
if len(files) <= 150:
    pass
elif len(files) > 150 and len(files) <= 400:
    for i,file in enumerate(files):
      if (i+1)%5 == 1:
        None
      else:
        # path = os.path.join(location_files, file)
        path = location_files + file
        os.remove(path)  # Indentation fixed here
        print("%s has been removed successfully" %file)

    #elif len(files) > 400 and len(files) <= 1000:
elif len(files) > 1000:  # Indentation fixed here
    for i,file in enumerate(files):
        path = os.path.join(directory, file)  # 'directory' variable needs to be defined
        os.remove(path)
        print("%s has been removed successfully" %file)
elif len(files) > 400:
    for i,file in enumerate(files):
        if (i+1)%8 == 1:
          None
        else:
          # path = os.path.join(location_files, file)
          path = location_files + file
          os.remove(path)
          print("%s has been removed successfully" %file)

    #path = os.path.join(location, file)  # This line seems unnecessary and might cause errors, consider removing it

In [18]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, datasets
import matplotlib.pyplot as plt


torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [19]:
mean = 0.2069
std = 0.1471

data_dir = '/content/drive/My Drive/al5083/al5083/train'
#data_dir = '/content/drive/MyDrive/alweld/train'
transform = transforms.Compose([
     transforms.CenterCrop((800,800)),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)
     ])

train_dataset = datasets.ImageFolder(root =data_dir,transform = transform)

original_labels_dict = train_dataset.class_to_idx

In [20]:
import json
# Opening JSON file
f = open('/content/drive/My Drive/al5083/al5083/train/train.json',)
#f = open('/content/drive/MyDrive/alweld/train/train.json',)

label_train = json.load(f)
keys_orig = list(label_train.keys())
values = list(label_train.values())
keys = []
for key in keys_orig:
  a_string = key
  a_string = a_string.replace("/", " ")
  x = a_string.split()
  string = x[0]+' '+x[1]
  keys.append(string)
true_labels_dict = {keys[i]: values[i] for i in range(len(keys))}
true_labels_dict

{'170906-113317-Al 2mm-part3': 1,
 '170906-144958-Al 2mm': 1,
 '170904-141730-Al 2mm-part2': 2,
 '170904-151845-Al 2mm-part2': 2,
 '170906-153326-Al 2mm-part2': 2,
 '170904-141232-Al 2mm-part2': 2,
 '170913-151508-Al 2mm-part2': 2,
 '170904-150144-Al 2mm-part1': 2,
 '170913-143933-Al 2mm-part2': 2,
 '170913-152931-Al 2mm-part2': 2,
 '170906-141809-Al 2mm-part2': 2,
 '170815-134756-Al 2mm': 2,
 '170904-115959-Al 2mm': 2,
 '170815-133921-Al 2mm': 2,
 '170904-112347-Al 2mm': 2,
 '170904-145718-Al 2mm-part2': 4,
 '170904-141730-Al 2mm-part3': 4,
 '170905-110711-Al 2mm-part2': 4,
 '170904-141232-Al 2mm-part3': 4,
 '170913-155806-Al 2mm-part2': 4,
 '170904-152301-Al 2mm-part1': 4,
 '170904-113012-Al 2mm-part2': 4,
 '170904-154202-Al 2mm-part2': 4,
 '170904-155523-Al 2mm': 4,
 '170913-140725-Al 2mm': 4,
 '170913-152931-Al 2mm-part1': 0,
 '170913-151508-Al 2mm-part1': 0,
 '170913-143933-Al 2mm-part1': 0,
 '170904-141730-Al 2mm-part1': 0,
 '170904-145718-Al 2mm-part1': 0,
 '170913-142501-Al 2mm

In [21]:
label_map_dict = {}

for key,value in original_labels_dict.items():
  label_map_dict.update({value:true_labels_dict[key]})
label_map_dict

{0: 2,
 1: 2,
 2: 2,
 3: 5,
 4: 4,
 5: 5,
 6: 2,
 7: 5,
 8: 2,
 9: 4,
 10: 0,
 11: 2,
 12: 4,
 13: 0,
 14: 4,
 15: 2,
 16: 5,
 17: 2,
 18: 4,
 19: 4,
 20: 4,
 21: 5,
 22: 4,
 23: 0,
 24: 0,
 25: 1,
 26: 0,
 27: 0,
 28: 0,
 29: 2,
 30: 1,
 31: 3,
 32: 3,
 33: 3,
 34: 5,
 35: 2,
 36: 4,
 37: 0,
 38: 0,
 39: 2,
 40: 0,
 41: 2,
 42: 0,
 43: 2,
 44: 0,
 45: 4}

In [22]:
true_labels_dict2 = {}
for key,value in true_labels_dict.items():
  if value ==0:
    val = 0
  else:
    val = 1
  dict2 = {key:val}
  true_labels_dict2.update(dict2)
true_labels_dict2

{'170906-113317-Al 2mm-part3': 1,
 '170906-144958-Al 2mm': 1,
 '170904-141730-Al 2mm-part2': 1,
 '170904-151845-Al 2mm-part2': 1,
 '170906-153326-Al 2mm-part2': 1,
 '170904-141232-Al 2mm-part2': 1,
 '170913-151508-Al 2mm-part2': 1,
 '170904-150144-Al 2mm-part1': 1,
 '170913-143933-Al 2mm-part2': 1,
 '170913-152931-Al 2mm-part2': 1,
 '170906-141809-Al 2mm-part2': 1,
 '170815-134756-Al 2mm': 1,
 '170904-115959-Al 2mm': 1,
 '170815-133921-Al 2mm': 1,
 '170904-112347-Al 2mm': 1,
 '170904-145718-Al 2mm-part2': 1,
 '170904-141730-Al 2mm-part3': 1,
 '170905-110711-Al 2mm-part2': 1,
 '170904-141232-Al 2mm-part3': 1,
 '170913-155806-Al 2mm-part2': 1,
 '170904-152301-Al 2mm-part1': 1,
 '170904-113012-Al 2mm-part2': 1,
 '170904-154202-Al 2mm-part2': 1,
 '170904-155523-Al 2mm': 1,
 '170913-140725-Al 2mm': 1,
 '170913-152931-Al 2mm-part1': 0,
 '170913-151508-Al 2mm-part1': 0,
 '170913-143933-Al 2mm-part1': 0,
 '170904-141730-Al 2mm-part1': 0,
 '170904-145718-Al 2mm-part1': 0,
 '170913-142501-Al 2mm

In [23]:
true_label_map_dict = {}

for key,value in original_labels_dict.items():
  true_label_map_dict.update({value:true_labels_dict2[key]})
true_label_map_dict

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 0,
 11: 1,
 12: 1,
 13: 0,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 0,
 24: 0,
 25: 1,
 26: 0,
 27: 0,
 28: 0,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 0,
 38: 0,
 39: 1,
 40: 0,
 41: 1,
 42: 0,
 43: 1,
 44: 0,
 45: 1}

In [24]:
#Setting of the main hyper-parameters of the model
batch_size = 200
#n_train = batch_size * 125    # Size of the train dataset
#n_test = batch_size * 25     # Size of the test dataset
n_channels = 4
initial_lr = 0.003          # Define your initial learning rate
num_epochs = 40

In [ ]:
# Define the quantum device and circuit
#n_qubits = 4  # Number of qubits to use
#dev = qml.device("default.qubit", wires=n_qubits)

#@qml.qnode(dev)
#def circuit(inputs, weights):
    #qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    #qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    #return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [64]:
@qml.qnode(dev)
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [86]:
class QuantumConv2D(nn.Module):
    """Quantum convolutional layer."""
    def __init__(self, n_qubits, n_output_channels, n_layers):
        super(QuantumConv2D, self).__init__()
        self.n_qubits = n_qubits
        self.n_output_channels = n_output_channels
        self.n_layers = n_layers
        # Initialize weights for the quantum circuit
        self.weights = nn.Parameter(
            torch.randn(n_layers, n_qubits, requires_grad=True)
        )

    def forward(self, x):
        # Quantum convolution operation
        batch_size, channels, height, width = x.size()
        x = x.permute(0, 2, 3, 1)  # Reshape to (batch_size, height, width, channels)

        out = torch.zeros((batch_size, height // 2, width // 2, self.n_output_channels), device=x.device)

        for b in range(batch_size):
            for i in range(0, height, 2):
                for j in range(0, width, 2):
                    # Apply quantum circuit to each 2x2 block
                    block = x[b, i:i+2, j:j+2, :].reshape(-1)
                    # Reduce the block size to match the number of qubits
                    if block.size(0) > self.n_qubits:
                        block = block[:self.n_qubits]  # Trim if more than n_qubits
                    elif block.size(0) < self.n_qubits:
                        block = F.pad(block, (0, self.n_qubits - block.size(0)))  # Pad if less than n_qubits

                    q_results = circuit(block.detach().numpy(), self.weights.detach().numpy())
                    q_results_tensor = torch.tensor([q.item() for q in q_results], dtype=torch.float32, device=x.device)

                    # Check the size of q_results_tensor and pad if necessary
                    if q_results_tensor.size(0) < self.n_output_channels:
                        q_results_tensor = F.pad(q_results_tensor, (0, self.n_output_channels - q_results_tensor.size(0)))
                    elif q_results_tensor.size(0) > self.n_output_channels:
                        q_results_tensor = q_results_tensor[:self.n_output_channels]

                    out[b, i // 2, j // 2, :] = q_results_tensor

        out = out.permute(0, 3, 1, 2)  # Reshape back to (batch_size, channels, height, width)
        return out



class QuantumNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            QuantumConv2D(n_qubits=4, n_output_channels=8, n_layers=4),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),

            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(8, 16, kernel_size=5, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),

            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),

            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 8, kernel_size=3),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(102400 // 200, 256),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.1),
            nn.Linear(128, 6),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x

def get_num_correct(pred, label):
    return pred.argmax(dim=1).eq(label).sum().item()

In [87]:
#train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
#optimizer = optim.Adam(QuantConv.parameters(), lr=5e-4, weight_decay = 1e-3)

#print(len(train_dataset))
#print(len(train_loader))

# Create an instance of the QuantModel
#model = QuantModel()
model = QuantumNetwork().to(device)
# Pass the model's parameters to the optimizer
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-3)

print(len(train_dataset))
print(len(train_loader))

5370
27


In [88]:
model.to(device)

QuantumNetwork(
  (features): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): QuantumConv2D()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0.1, inplace=False)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(8, 16, kernel_size=(5, 5), stride=(2, 2))
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
    (12): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Dropout(p=0.2, inplace=False)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16

In [89]:
%%time
# tic()
epoch = 0
acc = 0


model = QuantumNetwork()
model.to(device)


while epoch<10 and acc<90:
# for epoch in range(20): #
    total_loss = 0
    total_correct =0
    i=1
    total_length = 0
    for batch in train_loader:
        image, label_original = batch
        label = [true_label_map_dict[lab.item()] for lab in label_original]
        label = torch.tensor(label)
        # if i<10:


        pred = model(image.to(device))
        loss = F.cross_entropy(pred, label.to(device))
#         loss = F.mse_loss(pred, label.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(pred,label.to(device))
        total_length += len(label)
        acc = 100*total_correct/total_length
        print('Batch No.', i,'/',len(train_loader), ' Batch Accuracy :',  acc)
        i=i+1

    acc = total_correct*100/len(train_dataset)
    #-------------------------------------------------
    print('epoch: ', epoch+1, ', Total_correct: ', total_correct,', Loss: : ', total_loss, ' Accuracy: ', total_correct*100/len(train_dataset))
    epoch+=1
# print(toc())
# torch.set_grad_enabled(False)

KeyboardInterrupt: 

In [90]:
PATH_model = "/content/drive/My Drive/modelbetter.pt"
torch.save(model.state_dict(), PATH_model)

In [92]:
#test the model

test_data_dir = '/content/drive/My Drive/al5083/al5083/test/'

transform_test = transforms.Compose([
     transforms.CenterCrop((800,800)),
     transforms.ToTensor(),
     ])

test_dataset = datasets.ImageFolder(root =test_data_dir,transform = transform_test)
test_dataset

Dataset ImageFolder
    Number of datapoints: 6386
    Root location: /content/drive/My Drive/al5083/al5083/test/
    StandardTransform
Transform: Compose(
               CenterCrop(size=(800, 800))
               ToTensor()
           )

In [93]:
import json
# Opening JSON file
f = open('/content/drive/MyDrive/al5083/al5083/test/test.json',)

label_test = json.load(f)

In [94]:
keys_orig = list(label_test.keys())
values = list(label_test.values())
keys = []
for key in keys_orig:
  a_string = key
  a_string = a_string.replace("/", " ")
  x = a_string.split()
  string = x[0]+' '+x[1]
  keys.append(string)
true_test_labels_dict = {keys[i]: values[i] for i in range(len(keys))}
true_test_labels_dict

{'170906-143512-Al 2mm-part2': 1,
 '170904-155610-Al 2mm': 2,
 '170906-143512-Al 2mm-part1': 2,
 '170904-154202-Al 2mm-part1': 2,
 '170904-150144-Al 2mm-part2': 4,
 '170906-155007-Al 2mm-part2': 4,
 '170913-154448-Al 2mm': 0,
 '170906-113317-Al 2mm-part1': 0,
 '170905-112213-Al 2mm': 0,
 '170906-104925-Al 2mm': 0,
 '170904-152301-Al 2mm-part2': 5,
 '170906-153326-Al 2mm-part3': 3,
 '170906-151724-Al 2mm-part1': 3,
 '170906-155007-Al 2mm-part1': 3}

In [95]:
true_labels_dict3 = {}
for key,value in true_test_labels_dict.items():
  if value ==0:
    val = 0
  else:
    val = 1
  dict2 = {key:val}
  true_labels_dict3.update(dict2)
true_labels_dict3

{'170906-143512-Al 2mm-part2': 1,
 '170904-155610-Al 2mm': 1,
 '170906-143512-Al 2mm-part1': 1,
 '170904-154202-Al 2mm-part1': 1,
 '170904-150144-Al 2mm-part2': 1,
 '170906-155007-Al 2mm-part2': 1,
 '170913-154448-Al 2mm': 0,
 '170906-113317-Al 2mm-part1': 0,
 '170905-112213-Al 2mm': 0,
 '170906-104925-Al 2mm': 0,
 '170904-152301-Al 2mm-part2': 1,
 '170906-153326-Al 2mm-part3': 1,
 '170906-151724-Al 2mm-part1': 1,
 '170906-155007-Al 2mm-part1': 1}

In [96]:
original_test_labels_dict = test_dataset.class_to_idx
original_test_labels_dict

{'170904-150144-Al 2mm-part2': 0,
 '170904-152301-Al 2mm-part2': 1,
 '170904-154202-Al 2mm-part1': 2,
 '170904-155610-Al 2mm': 3,
 '170905-112213-Al 2mm': 4,
 '170906-104925-Al 2mm': 5,
 '170906-113317-Al 2mm-part1': 6,
 '170906-143512-Al 2mm-part1': 7,
 '170906-143512-Al 2mm-part2': 8,
 '170906-151724-Al 2mm-part1': 9,
 '170906-153326-Al 2mm-part3': 10,
 '170906-155007-Al 2mm-part1': 11,
 '170906-155007-Al 2mm-part2': 12,
 '170913-154448-Al 2mm': 13}

In [97]:
test_label_map_dict = {}

for key,value in original_test_labels_dict.items():
  test_label_map_dict.update({value:true_labels_dict3[key]})
test_label_map_dict

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 0,
 5: 0,
 6: 0,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 0}

In [98]:
torch.cuda.empty_cache()

torch.set_grad_enabled(False)

In [99]:
total_correct=0
correct_predictions = []
i=0
total = 0
test_loader = DataLoader(test_dataset, batch_size=200)
for batch in test_loader:
  image, label_original = batch
  label = [test_label_map_dict[lab.item()] for lab in label_original]
  label = torch.tensor(label)



  pred = model(image.to(device))

  total_correct += get_num_correct(pred,label.to(device))
  correct_predictions.append(total_correct)

  total += len(label)
  acc = 100*total_correct/(total)
  print('Batch No: ',i+1,', Test Batch Accuracy : ', acc)
  i=i+1
print(total_correct/len(test_dataset))

KeyboardInterrupt: 